In [1]:
import os
import cv2
import shutil
import tempfile
import tensorflow as tf
import subprocess

In [2]:
# get unichar and map accordingly
def getAlphabet(unichar_dir):
    num_cat = 2 + ord(u'\u007f')+1-ord(u'\u0021') + ord(u'\u0e7f')+1-ord(u'\u0e00')
    unichar_path = unichar_dir+'charset_size={}.txt'.format(num_cat)
    alphabet = u''
    with open(unichar_path, 'r', encoding = 'utf-8') as f:
        lines = f.readlines()
        for line in lines[:len(lines)-1]:
            alphabet += line.strip('\n').split('\t')[1]
    return alphabet, unichar_path

# def text_to_label(text):
#     if text=='<nul>':
#         return len(alphabet)
#     else:
#         return alphabet.find(text)

# def label_to_text(label):
#     if label == len(alphabet):
#         return '<nul>'
#     else:
#         return alphabet[label]

def cleanup(temp_dir):
    ''' Tries to remove the whole temp_dir '''
    try:
        shutil.rmtree(temp_dir)
    except OSError:
        pass

In [3]:
def inference(data_path, decoder_path, python_dir, train_dir, encoding='UTF-8', cuda_device=""):
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device
    os.environ["PYTHONIOENCODING"] = encoding
    
    os.chdir(python_dir)
    command = []
    os.environ
    command += ('python3', 'inference.py',
                '--image', data_path,
                '--decoder', decoder_path,
                '--train_dir', train_dir)
    result = subprocess.check_output(command)
    print(result.decode('utf-8'))
    
def my_predict(folder_dir):
    tempdir = tempfile.mkdtemp()
    # run inference
    inference(image_path, decoder_path, folder_dir+'python', folder_dir+'model')
    cleanup(tempdir)

In [ ]:
# choose the image for inference
image_path = '/data/test_iamge/tha+eng3.png'
# customized params
folder_dir = '/data/street/'
output_dir = '/data/'
unichar_dir = output_dir
alphabet, decoder_path = getAlphabet(unichar_dir)
# my model
import time
start = time.time()
my_predict(folder_dir)
print('spent: {:.2f}s'.format(time.time()-start))

In [ ]:
# tesseract output
import pytesseract
lang_choice='tha+eng'
tessdata_prefix = "/data/tesseract-ralated/tesseract-ocr/tessdata"
os.environ["TESSDATA_PREFIX"] = tessdata_prefix
start = time.time()
print(pytesseract.image_to_string(cv2.imread(image_path), lang=lang_choice))
print('spent: {:.2f}s'.format(time.time()-start))